# NO MACHIING LEARNING LIBRARY USED FOR TRAINING/PREDICTION/CLASSIFCATION

In [1]:
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.model_selection import train_test_split
import math
from nltk.corpus import stopwords

In [2]:
stop_words = stopwords.words('english')
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

# Why NLP?
combine manylabs with training('train.csv') daset
apply count vectorizer to both of them together
only train a model based on logistic regression using training dataset
use trained model to analyze sentiment of manylabs highpower and low power column
create a CSV FILE TO store sentiment for [highpower] [lowpoer] for each candidate

In [3]:
class Logistic:
    def __init__(self,rate,iteration):
        self.rate = rate
        self.weights = None
        self.iteration = iteration
    
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    
    def loss(self,h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def gradient_descent(self,X, h, y):
        return np.dot(X.T, (h - y)) / y.shape[0]

    def update_weight_loss(self,weight, learning_rate, gradient):
        return weight - learning_rate * gradient
    
    def initialize_parameters(self,X):
        self.weights = np.random.randn(len(X[0])) * 0.01
        
    def fit(self,X,y):
        self.initialize_parameters(X)
        #gradient descent
        for i in range(self.iteration):
            h = self.sigmoid(np.dot(X,self.weights))
            gradient = self.gradient_descent(X,h,y)
            self.weights = self.update_weight_loss(self.weights,self.rate,gradient)
            if i % 10 == 0:
                print('current iter is: ', i)
    def predict(self,X):
        ans = self.sigmoid(np.dot(X,self.weights))
        for i in range(len(ans)):
            if ans[i] < 0.5:
                ans[i] = 0
            else:
                ans[i] = 1
        return ans
    
    def predict_no_threshold(self,X):
        ans = self.sigmoid(np.dot(X,self.weights))
        ans_with_threshold = []
        for i in range(len(ans)):
            if(ans[i]) < 0.5 :
                ans_with_threshold.append(0)
            elif ans[i] == 0.5:
                ans_with_threshold.append(0.5)
            elif ans[i] > 0.5:
                ans_with_threshold.append(1)
        return ans,ans_with_threshold
    
    def test_accuracy(self,X,y):
        cor = 0
        total = len(y)
        for i in range(total):
            if X[i] == y[i]:
                cor+=1
        
        print("predict accuracy is: ", cor/total)
        return cor/total
        

In [4]:
def create_array_for_training_testing_sent_data():
    train = pd.read_csv('data/train.csv',encoding='latin-1')
    test = pd.read_csv('data/test.csv',encoding='latin-1')
    train = train[0:10000]
#     test = test[0:8000]
    #create list to hold all data
    train_tags = train['Sentiment'].values
    train_text = train['SentimentText'].values

    return train_text,train_tags

# CREATE df to store manylab text column

In [5]:
def many_lab_array(feature):
    df3 = pd.read_csv('./ML3AllSites.csv', sep=',',encoding='ISO-8859-1')
    df = pd.read_csv('./Tab.delimited.Cleaned.dataset.WITH.variable.labels.csv', sep='\t',encoding='ISO-8859-1')
    many_lab_text = df3[feature].values
    return many_lab_text
    

In [6]:
def processnull(arr):
    arrnew = []
    for i in range(len(arr)):
        if type(arr[i]) == float and math.isnan(arr[i]) :
            arrnew.append('nan')
        else:
            arrnew.append(arr[i])
    return arrnew

# Clean dataset into regular expression

In [7]:
def preprocess_reviews(reviews):

    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    NO_SPACE = ""
    SPACE = " "
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

# reviews_test_clean = preprocess_reviews(reviews_test)

# n-grams size = 1,2
# STOP_WORDS

In [14]:
def data_preprocessing(reviews_train_clean):
    stop_words = stopwords.words('english')
    ngram_vectorizer = CountVectorizer(binary=True,ngram_range=(1,1),stop_words= stop_words)
    ngram_vectorizer.fit(reviews_train_clean)
    X = ngram_vectorizer.transform(reviews_train_clean)
    return X
    

In [15]:
#append dataset into og
def append(og,lab_arr):
    ans = list(og)
    for i in range(len(lab_arr)):
        ans.append(lab_arr[i])
    return np.asarray(ans)

In [16]:
def recover_training_data(x,manylab):
    x_train = []
    x_pred = []
    for i in range(0,len(x)-len(manylab)):
        x_train.append(x[i])
    for i in range(len(x)-len(manylab),len(x)):
        x_pred.append(x[i])
    return np.asarray(x_train),np.asarray(x_pred)

# CREATE TRAINING AND PREDICTION DATASET

In [17]:
train_text,train_tags = create_array_for_training_testing_sent_data()
reviews_train_clean = preprocess_reviews(train_text)
man_arr = many_lab_array('lowpower')
man_arr_no_null = processnull(man_arr)
clean_man_arr = preprocess_reviews(man_arr_no_null)
new_x = append(reviews_train_clean,clean_man_arr)
X = data_preprocessing(new_x)
X_spase_many_lab = csr_matrix(X).toarray()
x_train,x_pred = recover_training_data(X_spase_many_lab,clean_man_arr)

/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (1,11,12,19,20,129,132,169,230) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (17,55,59,61,65,68,69,70,83,90,91,92,93,120,121,122,123,126,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


# TRAIN MODEL

In [18]:
log_with_many_lab = Logistic(0.1,100)

In [20]:
log_with_many_lab.fit(x_train,train_tags)

current iter is:  0
current iter is:  10
current iter is:  20
current iter is:  30
current iter is:  40
current iter is:  50
current iter is:  60
current iter is:  70
current iter is:  80
current iter is:  90


In [21]:
sentiment_analysis_for_many_labs,threshold = log_with_many_lab.predict_no_threshold(x_pred)

In [36]:
dic = []
for i in range(len(sentiment_analysis_for_many_labs)):
    if sentiment_analysis_for_many_labs[i] >0.505:
        dic.append(i)
print(dic)

[265, 921, 1535, 1579, 1921, 2096, 2154, 2274, 2319, 2337, 2387, 2430, 2800]


In [41]:
len(dic)

13

In [40]:
df3['lowpower'][1579]

"I was at an interview for a scholarship during high school, and the interviewer was asking me questions about my classes, grades and SATs. She also asked me what I saw myself doing in the future. I was nervous because at the time, I had classes that weren't my strengths and I felt they weren't a great representation of my abilities. I didn't feel confident and was nervous answering her questions."

In [34]:
df3 = pd.read_csv('./ML3AllSites.csv', sep=',',encoding='ISO-8859-1')
df3['lowpower'][2905]

/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,11,12,19,20,129,132,169,230) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'I was being evaluated at a sports tryout. I felt as if I had no power for the outcome of the tryout and felt very nervous not knowing how the personal in control felt about me making the team.'

In [101]:
pd.DataFrame(sentiment_analysis_for_many_labs).to_csv("./lower.csv",index=False,header=False)

# Sentiment Analysis for 'highpower'

In [42]:
train_text,train_tags = create_array_for_training_testing_sent_data()
reviews_train_clean = preprocess_reviews(train_text)
man_arr = many_lab_array('highpower')
man_arr_no_null = processnull(man_arr)
clean_man_arr = preprocess_reviews(man_arr_no_null)
new_x = append(reviews_train_clean,clean_man_arr)
X = data_preprocessing(new_x)
X_spase_many_lab = csr_matrix(X).toarray()
x_train,x_pred = recover_training_data(X_spase_many_lab,clean_man_arr)

/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (1,11,12,19,20,129,132,169,230) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (17,55,59,61,65,68,69,70,83,90,91,92,93,120,121,122,123,126,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [43]:
log_with_many_lab = Logistic(0.1,100)

In [ ]:
log_with_many_lab.fit(x_train,train_tags)

current iter is:  0


In [ ]:
sentiment_analysis_for_many_labs,threshold = log_with_many_lab.predict_no_threshold(x_pred)

In [ ]:
dic = []
for i in range(len(sentiment_analysis_for_many_labs)):
    if sentiment_analysis_for_many_labs[i] >0.505:
        dic.append(i)
print(dic)

# CODE BELOW ONLY USED FOR TESTING model performs on ORIGINAL sentiment dataset

In [13]:
stop_words = stopwords.words('english')
ngram_vectorizer = CountVectorizer(binary=True,ngram_range=(1,2),stop_words= stop_words)
train_text,train_tags = create_array_for_training_testing_sent_data()
reviews_train_clean = preprocess_reviews(train_text)
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)

In [14]:
x_train,x_pred,y_train,y_pred = train_test_split(X,train_tags,train_size = 0.75)

/usr/lib/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [15]:
def convert_sparse_to_np_array(x_train,y_train):
    return csr_matrix(x_train).toarray(),csr_matrix(y_train).toarray()  

In [16]:
x_train,x_pred = convert_sparse_to_np_array(x_train,x_pred)  #convert a scipy.sparse.csr.csr_matrix to np array

In [17]:
log_reg = Logistic(0.5,100)

In [18]:
x_train.shape

(7500, 75022)

In [19]:
log_reg.fit(x_train,y_train)

current iter is:  0
current iter is:  10
current iter is:  20
current iter is:  30
current iter is:  40
current iter is:  50
current iter is:  60
current iter is:  70
current iter is:  80
current iter is:  90


In [20]:
a_fial = log_reg.predict(x_pred)

In [21]:
log_reg.test_accuracy(a_fial,y_pred)

predict accuracy is:  0.7144


0.7144